In [1]:
!pip -q install rectools==0.2.0

In [2]:
!pip -q install optuna

In [3]:
!pip -q install hnswlib

In [4]:
import os

In [5]:
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS

In [6]:
# import warnings
# warnings.filterwarnings('ignore')

In [7]:
import pandas as pd
import numpy as np
import requests
import zipfile as zf
import optuna

import matplotlib.pyplot as plt
import seaborn as sns

import typing as tp
from tqdm import tqdm

from implicit.als import AlternatingLeastSquares
from lightfm import LightFM

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel

In [8]:
K_RECOS = 10
RANDOM_STATE = 101

# Первая часть задания

 Необходимо будет перебрать $N$ моделей $(N \geq 2)$ матричной факторизации и перебрать у них $K$ гиперпараметров $(K \geq 2)$ **(6 баллов)**
    - Для перебора гиперпараметров можно использовать [`Optuna`](https://github.com/optuna/optuna), [`Hyperopt`](https://github.com/hyperopt/hyperopt)

## LOAD DATA 

In [9]:
# download dataset by chunks
url = "https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip"

req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download: 100%|█████████▉| 78.6M/78.8M [00:09<00:00, 10.9MiB/s]

In [10]:
files = zf.ZipFile('kion_train.zip','r')
files.extractall()
files.close()

In [11]:
interactions = pd.read_csv('kion_train/interactions.csv')
users = pd.read_csv('kion_train/users.csv')
items = pd.read_csv('kion_train/items.csv')

Так как работа носит учебный характер возьмем часть (четверть) датасета, для того чтобы ускорить расчеты

In [12]:
users = users[:int(users.shape[0] / 4)].reset_index(drop=True)
interactions = interactions[interactions['user_id'].isin(users['user_id'])].reset_index(drop=True)
items = items[items['item_id'].isin(interactions['item_id'])].reset_index(drop=True)

Проверка

In [13]:
print(users.shape)
print(interactions.shape)
print(items.shape)

(210049, 5)
(1096495, 5)
(11288, 14)


## Preprocess

In [14]:
# rename columns
interactions = interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                                            'total_dur': Columns.Weight})

In [15]:
interactions = interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index)

In [16]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [17]:
max_date = interactions[Columns.Datetime].max()

In [18]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

Для теста оставим 2 недели, то есть 14 дней

In [19]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=14)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=14)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (922622, 5)
test: (173873, 5)


In [20]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [21]:
test = test.drop(test[test[Columns.User].isin(cold_users)].index)

## Prepare features

### User features

In [22]:
users.isnull().sum()

user_id        0
age         3529
income      3634
sex         3386
kids_flg       0
dtype: int64

In [23]:
users = users.fillna('Unknown')

In [24]:
users.nunique()

user_id     210049
age              7
income           7
sex              3
kids_flg         2
dtype: int64

In [25]:
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [26]:
users.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0


In [27]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


In [28]:
user_features.query(f"id == 973171")

,id,value,feature
0,973171,М,sex
0,973171,age_25_34,age
0,973171,income_60_90,income


### Item features

In [29]:
items.isnull().sum()

item_id             0
content_type        0
title               0
title_orig       2561
release_year       35
genres              0
countries          16
for_kids        10761
age_rating          1
studios         10590
directors         749
actors           1576
description         1
keywords          407
dtype: int64

In [30]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

In [31]:
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
4,11114,film,Принцесса Лебедь: Пират или принцесса,"The Swan Princess: Princess Tomorrow, Pirate T...",2016.0,"для детей, сказки, полнометражные, зарубежные,...",США,NaN,6.0,Sony Pictures,Ричард Рич,"Брайан Ниссен, Гарднер Джаэс, Грант Дураззо, Д...",Анимационная сказка о непоседливой принцессе Э...,"Принцесса, Лебедь, Пират, или, принцесса, 2016..."


In [32]:
items.nunique()

item_id         10923
content_type        2
title           10523
title_orig       8125
release_year      103
genres           2149
countries         616
for_kids            2
age_rating          6
studios            36
directors        6207
actors           9218
description     10717
keywords        10507
dtype: int64

#### Genre

In [33]:
# Explode genres to flatten table
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


#### Content

In [34]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

In [35]:
content_feature

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,16268,film,content_type
4,11114,film,content_type
...,...,...,...
11283,6443,series,content_type
11284,2367,series,content_type
11285,10632,series,content_type
11286,4538,series,content_type


#### Country

Добавим еще дополнительный признак "Страна выпуска" фильма

In [36]:
items["country"] = items["countries"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
country_feature = items[["item_id", "country"]].explode("country")
country_feature.columns = ["id", "value"]
country_feature["feature"] = "country"
country_feature.head()

,id,value,feature
0,10711,испания,country
1,2508,сша,country
2,10716,канада,country
3,16268,ссср,country
4,11114,сша,country


Объединим все признаки и посмотрим на один айтем, чтобы понимать, что получилось

In [37]:
item_features = pd.concat((genre_feature, content_feature, country_feature))

In [38]:
item_features[item_features['id'] ==  53]

,id,value,feature
2472,53,документальное,genre
2472,53,film,content_type
2472,53,франция,country


## Create dataset

In [39]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", 'content_type', "country"],
)

In [40]:
test_users = test[Columns.User].unique()

## Metrics

В качестве метрик будем рассматривать:
* Precision,
* Recall,
* MAP,

при k равном 10

In [41]:
metrics = {
    'precision@10': Precision(k=K_RECOS),
    'recall@10': Recall(k=K_RECOS),
    'MAP@10': MAP(k=K_RECOS),
}

Но при поиске будем учитывать только MAP метрику

## Models

Сначала подберем параметры с помощью `optuna` для рассматриваемых моделей `AlternatingLeastSquares`, `LightFM`, а затем сравним с "популярной моделью"

### AlternatingLeastSquares

In [42]:
def objective_implicit_ALS(trial):

    n_factors = trial.suggest_int("n_factors", low=64, high=128, step=32)
    regularization = trial.suggest_float("regularization", low=0.01, high=0.51, step=0.1)
    n_iterations = trial.suggest_int("n_iterationns", low=15, high=20, step=1)
    model_obj = ImplicitALSWrapperModel(
                  model=AlternatingLeastSquares(
                                        factors=n_factors, 
                                        regularization=regularization,
                                        iterations = n_iterations,
                                        random_state=RANDOM_STATE, 
                                                ),
                  fit_features_together=True,
                                        )

    model_obj.fit(dataset)
    
    recos = model_obj.recommend(
                                users=test_users,
                                dataset=dataset,
                                k=K_RECOS,
                                filter_viewed=True,
                                )
    
    metric_values = calc_metrics(metrics, recos, test, train)

    return metric_values['MAP@10']

In [43]:
%%time
study = optuna.create_study(directions=["maximize"])
study.optimize(objective_implicit_ALS, n_trials=10, n_jobs=-1)

[I 2022-12-12 07:15:53,064] A new study created in memory with name: no-name-9a79e5b5-0a64-4651-a228-d3295df69640
/usr/local/lib/python3.8/dist-packages/rectools/dataset/features.py:399: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")
/usr/local/lib/python3.8/dist-packages/rectools/models/implicit_als.py:269: UserWarning: GPU training requires number of factors to be a multiple of 32. Increasing factors from 260 to 288 (increasing latent factors from 64 to 92)
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/rectools/models/implicit_als.py:269: UserWarning: GPU training requires number of factors to be a multiple of 32. Increasing factors from 292 to 320 (increasing latent factors from 96 to 124)
  warnings.warn(
kion dataset download: 100%|██████████| 78.8M/78.8M [00:20<00:00, 10.9MiB/s][I 2022-12-12 07:18:12,978] Trial 1 finished with value: 0.08143870805914342 and p

CPU times: user 13min 42s, sys: 8min 9s, total: 21min 51s
Wall time: 11min 55s


In [44]:
study.best_trials

[FrozenTrial(number=3, values=[0.08339624395268729], datetime_start=datetime.datetime(2022, 12, 12, 7, 18, 25, 355898), datetime_complete=datetime.datetime(2022, 12, 12, 7, 20, 34, 695740), params={'n_factors': 64, 'regularization': 0.31000000000000005, 'n_iterationns': 15}, distributions={'n_factors': IntDistribution(high=128, log=False, low=64, step=32), 'regularization': FloatDistribution(high=0.51, log=False, low=0.01, step=0.1), 'n_iterationns': IntDistribution(high=20, log=False, low=15, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=3, state=TrialState.COMPLETE, value=None)]

Заносим полученные результаты

In [45]:
ALS = ImplicitALSWrapperModel(
                  model=AlternatingLeastSquares(
                                        factors=96, 
                                        regularization=0.41000000000000003,
                                        iterations = 15,
                                        random_state=RANDOM_STATE, 
                                                ),
                  fit_features_together=True,
                              )

### LightFm

In [46]:
def objective_LFM(trial):

    n_factors = trial.suggest_int("n_factors", low=64, high=128, step=32)
    loss = trial.suggest_categorical("loss", choices=['logistic', 'bpr', 'warp'])
    lr = trial.suggest_float("lr", low=0.05, high=0.25, step=0.05)

    model_obj = LightFMWrapperModel(
                  model=LightFM(
                                no_components=n_factors, 
                                loss=loss,
                                learning_rate=lr,
                                random_state=RANDOM_STATE,
                                ),
                  epochs=2,
                  num_threads=1
                                  )

    model_obj.fit(dataset)
    
    recos = model_obj.recommend(
                                users=test_users,
                                dataset=dataset,
                                k=K_RECOS,
                                filter_viewed=True,
                                )
    metric_values = calc_metrics(metrics, recos, test, train)

    return metric_values['MAP@10']

In [47]:
%%time
study = optuna.create_study(directions=["maximize"])
study.optimize(objective_LFM, n_trials=10, n_jobs=-1)

[I 2022-12-12 07:27:48,425] A new study created in memory with name: no-name-3c90437d-27df-48c2-bca8-28b56afc7528
[I 2022-12-12 07:29:21,268] Trial 1 finished with value: 0.0004547401680766152 and parameters: {'n_factors': 96, 'loss': 'logistic', 'lr': 0.15000000000000002}. Best is trial 1 with value: 0.0004547401680766152.
[I 2022-12-12 07:30:01,574] Trial 0 finished with value: 0.014967926810575882 and parameters: {'n_factors': 128, 'loss': 'warp', 'lr': 0.15000000000000002}. Best is trial 0 with value: 0.014967926810575882.
[I 2022-12-12 07:30:46,903] Trial 2 finished with value: 0.017089190090247668 and parameters: {'n_factors': 64, 'loss': 'warp', 'lr': 0.15000000000000002}. Best is trial 2 with value: 0.017089190090247668.
[I 2022-12-12 07:31:46,257] Trial 3 finished with value: 0.06439811859175822 and parameters: {'n_factors': 96, 'loss': 'warp', 'lr': 0.1}. Best is trial 3 with value: 0.06439811859175822.
[I 2022-12-12 07:32:58,996] Trial 4 finished with value: 0.02815544494813

CPU times: user 12min 53s, sys: 4min 43s, total: 17min 36s
Wall time: 9min 10s


In [48]:
study.best_trials

[FrozenTrial(number=6, values=[0.0763618029687734], datetime_start=datetime.datetime(2022, 12, 12, 7, 32, 59, 4655), datetime_complete=datetime.datetime(2022, 12, 12, 7, 34, 37, 493221), params={'n_factors': 96, 'loss': 'warp', 'lr': 0.05}, distributions={'n_factors': IntDistribution(high=128, log=False, low=64, step=32), 'loss': CategoricalDistribution(choices=('logistic', 'bpr', 'warp')), 'lr': FloatDistribution(high=0.25, log=False, low=0.05, step=0.05)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=6, state=TrialState.COMPLETE, value=None)]

Заносим полученные результаты

In [49]:
LFM = LightFMWrapperModel(
                model=LightFM(
                              no_components=64, 
                              loss='warp',
                              learning_rate=0.05,
                              random_state=RANDOM_STATE,
                              ),
                epochs=2,
                num_threads=1
                          )

### Сравнение моделей

In [50]:
models = {
    'popular': PopularModel(),
    'ALS': ALS,
    'LFM': LFM
}

In [51]:
models

{'popular': <rectools.models.popular.PopularModel at 0x7faafaebcaf0>,
 'ALS': <rectools.models.implicit_als.ImplicitALSWrapperModel at 0x7faafaebc340>,
 'LFM': <rectools.models.lightfm.LightFMWrapperModel at 0x7faafaeac610>}

In [52]:
%%time
results = []
for model_name, model in models.items():
    print(f"Fitting model {model_name}...")
    model_quality = {'model': model_name}

    model.fit(dataset)
    recos = model.recommend(
        users=test_users,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    metric_values = calc_metrics(metrics, recos, test, train)
    model_quality.update(metric_values)
    results.append(model_quality)

Fitting model popular...
Fitting model ALS...


/usr/local/lib/python3.8/dist-packages/rectools/dataset/features.py:399: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")
/usr/local/lib/python3.8/dist-packages/rectools/models/implicit_als.py:269: UserWarning: GPU training requires number of factors to be a multiple of 32. Increasing factors from 292 to 320 (increasing latent factors from 96 to 124)
  warnings.warn(


Fitting model LFM...
CPU times: user 1min 37s, sys: 26.8 s, total: 2min 4s
Wall time: 1min 16s


In [53]:
df_quality = pd.DataFrame(results).T
df_quality.columns = df_quality.iloc[0]
df_quality = df_quality.drop('model')

In [54]:
df_quality.style.highlight_max(color='red', axis=1)

model,popular,ALS,LFM
precision@10,0.039748,0.039314,0.039940
recall@10,0.161997,0.158501,0.156127
MAP@10,0.080036,0.082977,0.076772


### Вывод

Как видно из таблицы, лучше на кусочке датасета справилась модель ALS. На это также повлиял подбор гиперпараметров моделей, учивая учебный характер работы, количество итераций и диапазон параметров выбирался, чтобы минимизировать время подбора и уложится в 15минут.

---

# Вторая часть задания

- Воспользоваться методом приближенного поиска соседей для выдачи рекомендаций. **(3 балла)**

## Approximate Nearest Neighbors 

In [55]:
import nmslib

## HNSW algorithm parameters

### Search parameters:
* ```ef``` - the size of the dynamic list for the nearest neighbors (used during the search). Higher ```ef```
leads to more accurate but slower search. ```ef``` cannot be set lower than the number of queried nearest neighbors
```k```. The value ```ef``` of can be anything between ```k``` and the size of the dataset.
* ```k``` number of nearest neighbors to be returned as the result.
The ```knn_query``` function returns two numpy arrays, containing labels and distances to the k found nearest 
elements for the queries. Note that in case the algorithm is not be able to find ```k``` neighbors to all of the queries,
(this can be due to problems with graph or ```k```>size of the dataset) an exception is thrown.


### Construction parameters:
* ```M``` - the number of bi-directional links created for every new element during construction. Reasonable range for ```M``` 
is 2-100. Higher ```M``` work better on datasets with high intrinsic dimensionality and/or high recall, while low ```M``` work 
better for datasets with low intrinsic dimensionality and/or low recalls. The parameter also determines the algorithm's memory 
consumption, which is roughly ```M * 8-10``` bytes per stored element.  
As an example for ```dim```=4 random vectors optimal ```M``` for search is somewhere around 6, while for high dimensional datasets 
(word embeddings, good face descriptors), higher ```M``` are required (e.g. ```M```=48-64) for optimal performance at high recall. 
The range ```M```=12-48 is ok for the most of the use cases. When ```M``` is changed one has to update the other parameters. 
Nonetheless, ef and ef_construction parameters can be roughly estimated by assuming that ```M```*```ef_{construction}``` is 
a constant.

* ```ef_construction``` - the parameter has the same meaning as ```ef```, but controls the index_time/index_accuracy. Bigger 
ef_construction leads to longer construction, but better index quality. At some point, increasing ef_construction does
not improve the quality of the index. One way to check if the selection of ef_construction was ok is to measure a recall 
for M nearest neighbor search when ```ef``` =```ef_construction```: if the recall is lower than 0.9, than there is room 
for improvement.
* ```num_elements``` - defines the maximum number of elements in the index. The index can be extened by saving/loading(load_index
function has a parameter which defines the new maximum number of elements).

Еще источники: 
- [Nmslib Docs](https://github.com/nmslib/nmslib/blob/master/manual/methods.md)
- [Pinecone Vector Indexes](https://www.pinecone.io/learn/vector-indexes/)

<img src="https://d33wubrfki0l68.cloudfront.net/4c635fabb268a4af60109a506300a2dfda612063/d2535/images/similarity-search-indexes17.jpg">

<img src="https://d33wubrfki0l68.cloudfront.net/96d80cd46c2d12df99c044c860a8a5fb00cf6376/d59ca/images/similarity-search-indexes18.jpg">

In [56]:
import time

In [57]:
model

In [58]:
user_embeddings, item_embeddings = model.get_vectors(dataset)

In [59]:
user_embeddings.shape, item_embeddings.shape

((164028, 66), (10923, 66))

In [60]:
def augment_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()
    
    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [61]:
print('pre shape: ', item_embeddings.shape)
max_norm, augmented_item_embeddings = augment_inner_product(item_embeddings)
augmented_item_embeddings.shape

pre shape:  (10923, 66)


(10923, 67)

In [62]:
extra_zero = np.zeros((user_embeddings.shape[0], 1))
augmented_user_embeddings = np.append(user_embeddings, extra_zero, axis=1)
augmented_user_embeddings.shape

(164028, 67)

In [63]:
user_id = 30

In [64]:
user_embeddings[user_id]

array([-1.59771423e+02,  1.00000000e+00, -5.56216165e-02, -4.64792997e-02,
        1.30138129e-01,  1.53311104e-01,  2.34122545e-01, -2.53335148e-01,
        3.33680995e-02,  1.46806359e-01,  2.45155215e-01,  1.31152704e-01,
       -1.31818563e-01,  6.23928010e-03,  2.47402415e-02, -5.68863526e-02,
       -2.47519210e-01, -2.54685462e-01,  1.59820303e-01,  1.54541016e-01,
        1.56995028e-01,  1.51237458e-01, -1.91981524e-01,  2.82042801e-01,
        2.33061612e-04, -1.60121247e-01,  3.03198516e-01, -3.93469661e-01,
        5.70290610e-02, -3.37728262e-01,  8.29877555e-02,  1.03017211e-01,
       -3.11132133e-01,  2.65858740e-01,  1.28414810e-01,  7.21396208e-02,
       -2.07272410e-01,  5.64156659e-03,  6.42190427e-02,  6.43020198e-02,
        1.33041851e-02, -7.25318864e-02, -3.90876532e-02, -9.16699022e-02,
        1.33536205e-01, -9.66947824e-02,  1.82479575e-01, -3.23576510e-01,
       -7.55766332e-02, -9.22435671e-02,  1.12079725e-01,  7.36328214e-03,
       -1.09877594e-01, -

In [65]:
augmented_user_embeddings[user_id]

array([-1.59771423e+02,  1.00000000e+00, -5.56216165e-02, -4.64792997e-02,
        1.30138129e-01,  1.53311104e-01,  2.34122545e-01, -2.53335148e-01,
        3.33680995e-02,  1.46806359e-01,  2.45155215e-01,  1.31152704e-01,
       -1.31818563e-01,  6.23928010e-03,  2.47402415e-02, -5.68863526e-02,
       -2.47519210e-01, -2.54685462e-01,  1.59820303e-01,  1.54541016e-01,
        1.56995028e-01,  1.51237458e-01, -1.91981524e-01,  2.82042801e-01,
        2.33061612e-04, -1.60121247e-01,  3.03198516e-01, -3.93469661e-01,
        5.70290610e-02, -3.37728262e-01,  8.29877555e-02,  1.03017211e-01,
       -3.11132133e-01,  2.65858740e-01,  1.28414810e-01,  7.21396208e-02,
       -2.07272410e-01,  5.64156659e-03,  6.42190427e-02,  6.43020198e-02,
        1.33041851e-02, -7.25318864e-02, -3.90876532e-02, -9.16699022e-02,
        1.33536205e-01, -9.66947824e-02,  1.82479575e-01, -3.23576510e-01,
       -7.55766332e-02, -9.22435671e-02,  1.12079725e-01,  7.36328214e-03,
       -1.09877594e-01, -

In [66]:
item_id = 0

In [67]:
item_embeddings[item_id]

array([ 1.00000000e+00, -8.97430778e-01, -3.00629884e-01, -1.01674497e+00,
       -4.35178339e-01, -2.09030986e-01, -3.59013408e-01,  6.52885854e-01,
       -1.96654901e-01,  2.90212810e-01, -2.26102531e-01, -5.38863800e-02,
        2.69196838e-01,  6.87581375e-02,  9.16260362e-01,  5.23185909e-01,
       -3.42100337e-02,  9.84004810e-02, -1.16481781e+00,  2.37992644e-01,
        2.47852355e-01,  7.61459827e-01,  1.78273469e-01, -1.07198167e+00,
        6.77900910e-02,  5.23971379e-01, -1.98327973e-01,  5.73036313e-01,
       -9.46981668e-01,  2.22669542e-01, -2.56788641e-01,  1.39433071e-02,
        1.81689411e-01,  1.30118108e+00,  8.75966907e-01,  3.21544036e-02,
        4.59409148e-01,  2.79437542e-01, -6.23002172e-01, -1.37834978e+00,
       -3.73366326e-01,  7.34697700e-01, -5.77590466e-01,  1.90247715e-01,
       -5.78223646e-01,  4.44867343e-01,  3.79820079e-01, -5.34953952e-01,
        6.81248903e-01,  5.91714904e-02,  2.74457633e-01, -6.28490806e-01,
        2.12341380e+00, -

In [68]:
augmented_item_embeddings[item_id]

array([ 1.00000000e+00, -8.97430778e-01, -3.00629884e-01, -1.01674497e+00,
       -4.35178339e-01, -2.09030986e-01, -3.59013408e-01,  6.52885854e-01,
       -1.96654901e-01,  2.90212810e-01, -2.26102531e-01, -5.38863800e-02,
        2.69196838e-01,  6.87581375e-02,  9.16260362e-01,  5.23185909e-01,
       -3.42100337e-02,  9.84004810e-02, -1.16481781e+00,  2.37992644e-01,
        2.47852355e-01,  7.61459827e-01,  1.78273469e-01, -1.07198167e+00,
        6.77900910e-02,  5.23971379e-01, -1.98327973e-01,  5.73036313e-01,
       -9.46981668e-01,  2.22669542e-01, -2.56788641e-01,  1.39433071e-02,
        1.81689411e-01,  1.30118108e+00,  8.75966907e-01,  3.21544036e-02,
        4.59409148e-01,  2.79437542e-01, -6.23002172e-01, -1.37834978e+00,
       -3.73366326e-01,  7.34697700e-01, -5.77590466e-01,  1.90247715e-01,
       -5.78223646e-01,  4.44867343e-01,  3.79820079e-01, -5.34953952e-01,
        6.81248903e-01,  5.91714904e-02,  2.74457633e-01, -6.28490806e-01,
        2.12341380e+00, -

In [69]:
# Set index parameters
# These are the most important ones
M = 48
efC = 100

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}
print('Index-time parameters', index_time_params)

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100, 'post': 0}


In [70]:
# Number of neighbors 
K=10

In [71]:
# Space name should correspond to the space name 
# used for brute-force search
space_name='negdotprod'

In [72]:
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(augmented_item_embeddings) 

10923

In [73]:
index

<nmslib.FloatIndex method='hnsw' space='negdotprod' at 0x7457d800>

In [74]:
# Create an index
start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100}
Indexing time = 0.591288


In [75]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 100}


In [76]:
query_matrix = augmented_user_embeddings[:1000, :]

In [77]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

kNN time total=0.045512 (sec), per query=0.000046 (sec), per query adjusted for thread number=0.000182 (sec)


In [78]:
nbrs[0]

(array([10683,  2094,  2962,  5044,  1938,  9491,  8568,  4401,  7759,
         8607], dtype=int32),
 array([155.18996, 155.45961, 155.60623, 155.71233, 155.71727, 155.72987,
        155.77228, 155.81204, 155.8491 , 155.85019], dtype=float32))

In [79]:
nbrs[0][1]

array([155.18996, 155.45961, 155.60623, 155.71233, 155.71727, 155.72987,
       155.77228, 155.81204, 155.8491 , 155.85019], dtype=float32)

In [80]:
def recommend_all(query_factors, index_factors, topn=10):
    output = query_factors.dot(index_factors.T)
    argpartition_indices = np.argpartition(output, -topn)[:, -topn:]

    x_indices = np.repeat(np.arange(output.shape[0]), topn)
    y_indices = argpartition_indices.flatten()
    top_value = output[x_indices, y_indices].reshape(output.shape[0], topn)
    top_indices = np.argsort(top_value)[:, ::-1]

    y_indices = top_indices.flatten()
    top_indices = argpartition_indices[x_indices, y_indices]
    labels = top_indices.reshape(-1, topn)
    distances = output[x_indices, top_indices].reshape(-1, topn)
    return labels, distances

In [81]:
recommend_all(user_embeddings[[0], :], item_embeddings)

(array([[10683,  2094,  2962,  5044,  1938,  9491,  8568,  4401,  7759,
          8607]]),
 array([[-155.18994964, -155.4595988 , -155.60621778, -155.71230646,
         -155.71728697, -155.72989191, -155.7722715 , -155.81207334,
         -155.84909091, -155.85017543]]))

In [82]:
item_embeddings[:1000, :].shape, user_embeddings.shape

((1000, 66), (164028, 66))

In [83]:
query_matrix_not_augmented = user_embeddings[:1000, :]

In [84]:
%%timeit
recommend_all(query_matrix_not_augmented, item_embeddings)

223 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
%%timeit
index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)

33.3 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [86]:
import hnswlib

In [87]:
%%time
max_elements, dim = augmented_item_embeddings.shape
hnsw = hnswlib.Index("ip", dim) # possible options for space are l2, cosine or ip

# Initing index - the maximum number of elements should be known beforehand
hnsw.init_index(max_elements, M, efC)

# Element insertion (can be called several times)
hnsw.add_items(augmented_item_embeddings)

CPU times: user 1.07 s, sys: 6.24 ms, total: 1.08 s
Wall time: 558 ms


In [88]:
# Controlling the recall by setting ef, should always be > k
hnsw.set_ef(efS)

In [89]:
label, distance = hnsw.knn_query(query_matrix, k=K)

In [90]:
label

array([[10683,  2094,  2962, ...,  4401,  7759,  8607],
       [10129,  6891,  6425, ...,  9186,  3231,    95],
       [ 8063,  2939,  1132, ...,  8607,  4899,  5008],
       ...,
       [ 6425,  6891,  3769, ...,  3231,  5044, 10239],
       [ 9186,  6891, 10129, ...,    95,  4706,  5714],
       [ 6891, 10129,  2739, ...,  8059,  6615,  4088]], dtype=uint64)

In [91]:
1 - distance

array([[-155.19   , -155.45958, -155.60622, ..., -155.81204, -155.84909,
        -155.85019],
       [-114.38053, -114.58346, -114.68692, ..., -115.26044, -115.35094,
        -115.6058 ],
       [-169.90651, -170.3952 , -170.61255, ..., -170.85808, -170.89005,
        -170.89322],
       ...,
       [-144.39438, -144.60883, -145.01569, ..., -145.22995, -145.31606,
        -145.38011],
       [-154.47566, -154.67354, -154.76929, ..., -155.19647, -155.28358,
        -155.3242 ],
       [-140.59355, -140.66124, -141.54552, ..., -142.3259 , -142.33812,
        -142.37802]], dtype=float32)

In [92]:
item_embeddings[8867].dot(user_embeddings[0])

-161.11325865570245

In [93]:
labels, distances = recommend_all(user_embeddings[:1000, :], item_embeddings)
print(labels)
print(distances)

[[10683  2094  2962 ...  4401  7759  8607]
 [10129  6891  6425 ...  9186  3231    95]
 [ 8063  2939  1132 ...  8607  4899  5008]
 ...
 [ 6425  6891  3769 ...  3231  5044 10239]
 [ 9186  6891 10129 ...    95  4706  5714]
 [ 6891 10129  2739 ...  8059  6615  4088]]
[[-155.18994964 -155.4595988  -155.60621778 ... -155.81207334
  -155.84909091 -155.85017543]
 [-114.38052433 -114.58345858 -114.68692821 ... -115.26043798
  -115.35094009 -115.60580014]
 [-169.90649235 -170.39519083 -170.61255039 ... -170.85808215
  -170.89002471 -170.89318512]
 ...
 [-144.39438751 -144.60880903 -145.01569837 ... -145.22997086
  -145.31603655 -145.38011923]
 [-154.47566221 -154.67353641 -154.76926128 ... -155.19650198
  -155.28359009 -155.3242225 ]
 [-140.59357498 -140.6612127  -141.54553228 ... -142.3258788
  -142.33812027 -142.37800599]]


#Третья часть задания

- Добавить 3 "аватаров" (искусственных пользователей) и посмотреть рекомендации итоговой модели на них. Объяснить почему добавили именно таких пользователей. **(3 балла)**

Создадим трех аватаров:
* `666` - Мужщина, в возрасте 35-44, с доходом 60-90, с детьми, который любит смотреть фантастику
* `999` - Женщина, в возрасте 25-34, с доходом 60-90, без детей, которая любит смотреть драмы
* `777` - Мужщина, в возрасте 25-34, с доходом 60-90, без детей, который любит смотреть документалки

Выбрал именно, этих пользователей, потому что на мой взгляд они очень "типичны" для нашего общества.

Проверим, что их нет в нашем датасете

In [94]:
np.in1d(np.array([666, 777, 999]), users['user_id'].unique())

array([False, False, False])

Теперь добавим их в соответствующие таблицы + заново соберем датасет. И посмотрим, что предскажет наша лучшая модель, то есть ALS

In [95]:
user_666 = pd.DataFrame([[666, 'М', 'sex'], 
                         [666, 'age_35_44', 'age'], 
                         [666, 'income_60_90', 'income']], 
                        columns = user_features.columns
                        )
user_999 = pd.DataFrame([[999, 'Ж', 'sex'], 
                         [999, 'age_25_34', 'age'], 
                         [999, 'income_60_90', 'income']], 
                        columns = user_features.columns
                        )
user_777 = pd.DataFrame([[777, 'М', 'sex'], 
                         [777, 'age_25_34', 'age'], 
                         [777, 'income_60_90', 'income']], 
                        columns = user_features.columns
                        )

In [96]:
user_features = pd.concat((user_features, user_666, user_777, user_999))
user_features = user_features.reset_index(drop=True)

Проверка

In [97]:
user_features

,id,value,feature
0,973171,М,sex
1,962099,М,sex
2,721985,Ж,sex
3,704055,Ж,sex
4,1037719,М,sex
...,...,...,...
492088,777,age_25_34,age
492089,777,income_60_90,income
492090,999,Ж,sex
492091,999,age_25_34,age


Теперь в таблицу `train` добавим просмотры данных аватаров

In [98]:
fantasy_array = item_features.query('value == "фантастика"')['id'].head(10).values

In [99]:
items[items['item_id'].isin(fantasy_array)]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre,country
27,7308,film,Дивергент,Divergent,2014.0,"боевики, фантастика, детективы, мелодрамы",США,NaN,12.0,NaN,Нил Бёргер,"Эшли Джадд, Тони Голдуин, Мэгги Кью, Кейт Уинс...",В антиутопическом Чикаго будущего существует о...,"по роману или книге, будущее, антиутопия, футу...","[боевики, фантастика, детективы, мелодрамы]",[сша]
28,4358,film,Тело,Replace,2017.0,"фантастика, триллеры","Германия, Канада",NaN,18.0,NaN,Норберт Кайль,"Ребекка Форсайт, Люси Арон, Барбара Крэмптон, ...","Молодая женщина Кира мало того, что страдает п...","хоррор про тело, Бюстгальтер, Девушка-подросто...","[фантастика, триллеры]","[германия, канада]"
36,149,film,Разлом времени,Curvature,2017.0,"фантастика, триллеры, детективы",США,NaN,18.0,NaN,Диего Халливис,"Гленн Моршауэр, Зак Эйвери, Линда Хэмилтон, Ли...",Талантливый инженер Хелен тяжело переживает са...,"путешествие во времени, Автобус, Бейсбол, Бомб...","[фантастика, триллеры, детективы]",[сша]
65,2074,series,Посредник,Posrednik,1990.0,"боевики, фантастика",Россия,NaN,16.0,NaN,В.Потапов,"Олеся Судзиловская, Инара Слуцка, Валерий Стор...",Нависший над землей загадочный шар становится ...,"инопланетянин, открытый космос","[боевики, фантастика]",[россия]
79,10017,film,Парадокс Элис,Paradox Alice,2012.0,"фантастика, зарубежные, триллеры",США,NaN,18.0,NaN,Эрик Дапкевич,"Джек Брэнд, Дженета Ст. Клэр, Итан Шарретт, Ки...",Экипаж астронавтов отправляется на заледенелый...,"Парадокс, Элис, 2012, США, конец, света, космо...","[фантастика, зарубежные, триллеры]",[сша]
84,12191,film,Назад в будущее. Часть 2,Back to the Future Part II,1989.0,"приключения, зарубежные, фантастика, боевики, ...",США,NaN,12.0,Universal,Роберт Земекис,"Билли Зейн, Дж.Дж. Коэн, Джей Кох, Джеймс Толк...","Марти МакФлай только что вернулся из прошлого,...","Назад, будущее, Часть, 2, 1989, США, безумные,...","[приключения, зарубежные, фантастика, боевики,...",[сша]
98,2817,film,Охотники за привидениями,Ghostbusters,1984.0,"боевики, фантастика, фэнтези, комедии",США,NaN,6.0,NaN,Айвен Райтман,"Билл Мюррэй, Дэн Эйкройд, Сигурни Уивер, Уилья...","В конце двадцатого века оказывается, что в Нью...","библиотека, мифология, неудачник, зефир, слизь...","[боевики, фантастика, фэнтези, комедии]",[сша]
101,4723,film,Притяжение,Prityazhenie,2017.0,фантастика,Россия,NaN,12.0,NaN,Фёдор Бондарчук,"Олег Меньшиков, Никита Тарасов, Татьяна Шитова...","…Как только что стало известно, сбитый над Мос...","Россия, НЛО, инопланетянин, инопланетный кораб...",[фантастика],[россия]
105,3509,film,Комната желаний,The Room,2019.0,"драмы, фантастика, триллеры",Франция,NaN,16.0,NaN,Кристиан Волькман,"Ольга Куриленко, Кевин Янссенс, Джошуа Уилсон,...",Влюбленная пара решает переехать в уединенный ...,"психологический триллер, семья, реальность про...","[драмы, фантастика, триллеры]",[франция]
110,6720,film,Терминатор,"TERMINATOR, THE",1984.0,"боевики, фантастика, триллеры","Великобритания, США",NaN,16.0,NaN,Джеймс Кэмерон,"Арнольд Шварценеггер, Майкл Бин, Линда Хэмилто...",История противостояния солдата Кайла Риза и ки...,"спасение мира, искусственный интеллект, повста...","[боевики, фантастика, триллеры]","[великобритания, сша]"


In [100]:
inter_user_666 = pd.DataFrame({'user_id': [666]*10,
                               'item_id': fantasy_array,
                               'datetime': train['datetime'].sample(10).values,
                               'weight': [3]*10,
                               'watched_pct':[100.]*10}
                              )
inter_user_666

,user_id,item_id,datetime,weight,watched_pct
0,666,7308,2021-07-26,3,100.0
1,666,4358,2021-03-17,3,100.0
2,666,149,2021-08-01,3,100.0
3,666,2074,2021-06-20,3,100.0
4,666,10017,2021-04-25,3,100.0
5,666,12191,2021-07-10,3,100.0
6,666,2817,2021-04-05,3,100.0
7,666,4723,2021-03-14,3,100.0
8,666,3509,2021-04-25,3,100.0
9,666,6720,2021-05-16,3,100.0


Следующий аватар

In [101]:
drama_array = item_features.query('value == "мелодрамы"')['id'].head(10).values

In [102]:
items[items['item_id'].isin(drama_array)]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre,country
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...","[драмы, зарубежные, детективы, мелодрамы]",[испания]
3,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...","[драмы, спорт, советские, мелодрамы]",[ссср]
6,8604,film,Третья попытка,NaN,2013.0,"русские, мелодрамы",Россия,NaN,12.0,NaN,Игорь Мужжухин,"Александр Асташенок, Александр Пашков, Андрей ...","Екатерина Рябова, Александр Асташенок и Алекса...","Третья, попытка, 2013, Россия, любовь, измена,...","[русские, мелодрамы]",[россия]
7,3526,film,Код «Красный»,Red Joan,2018.0,"биография, экранизации, драмы, зарубежные, мел...",Великобритания,NaN,18.0,NaN,Тревор Нанн,"Бен Майлз, Джуди Денч, Лоуренс Спэллман, Софи ...",Тихая английская пенсионерка Джоан попадает по...,"Код, Красный, 2018, Великобритания, друзья, лю...","[биография, экранизации, драмы, зарубежные, ме...",[великобритания]
14,13109,film,Новый парень моей мамы,My Mom's New Boyfriend,2007.0,"мелодрамы, зарубежные, криминал, комедии",Германия,NaN,12.0,NaN,Джордж Галло,"Антонио Бандерас, Джон Вальдетеро, Кит Дэвид, ...",«Новый парень моей мамы» – американо-германска...,"Новый, парень, моей, мамы, 2007, Германия, огр...","[мелодрамы, зарубежные, криминал, комедии]",[германия]
18,14986,film,Год золотой рыбки,Goldfish Year,2007.0,"русские, мелодрамы",Украина,NaN,16.0,NaN,Андрей Красавин,"Александр Самойленко, Елена Панова, Дмитрий Ул...",У очаровательной певицы Лады успех во всех жиз...,"Год, золотой, рыбки, 2007, Украина","[русские, мелодрамы]",[украина]
27,7308,film,Дивергент,Divergent,2014.0,"боевики, фантастика, детективы, мелодрамы",США,NaN,12.0,NaN,Нил Бёргер,"Эшли Джадд, Тони Голдуин, Мэгги Кью, Кейт Уинс...",В антиутопическом Чикаго будущего существует о...,"по роману или книге, будущее, антиутопия, футу...","[боевики, фантастика, детективы, мелодрамы]",[сша]
33,5780,film,Теория хаоса,Chaos Theory,2008.0,"драмы, мелодрамы, комедии",США,NaN,12.0,NaN,Маркос Сига,"Райан Рейнольдс, Эмили Мортимер, Стюарт Таунсе...","История о Фрэнке, славном парне, одержимом кар...","драка, нижнее белье, медицинский тест, жених, ...","[драмы, мелодрамы, комедии]",[сша]
38,6844,film,Голоса за кадром,Golden Voices,2019.0,"драмы, мелодрамы, комедии",Израиль,NaN,16.0,NaN,Евгений Руман,"Мария Белкина, Владимир Фридман, Эвелин Хагоэл...","«Золотые голоса» героев, известных актеров дуб...","христианский фильм, 2019, израиль, голоса, за,...","[драмы, мелодрамы, комедии]",[израиль]
39,830,film,Бетховен 2,Beethoven's 2nd,1993.0,"мелодрамы, семейное, комедии",США,NaN,6.0,NaN,Род Дэниэл,"Чарльз Гродин, Бонни Хант, Николь Том, Кристоф...","Псу, случайно названному именем великого компо...","калифорния, холдинги, продажа, щенок, собака, ...","[мелодрамы, семейное, комедии]",[сша]


In [103]:
inter_user_999 = pd.DataFrame({'user_id': [999]*10,
                               'item_id': drama_array,
                               'datetime': train['datetime'].sample(10).values,
                               'weight': [3]*10,
                               'watched_pct':[100.]*10}
                              )
inter_user_999

,user_id,item_id,datetime,weight,watched_pct
0,999,10711,2021-04-17,3,100.0
1,999,16268,2021-07-17,3,100.0
2,999,8604,2021-05-22,3,100.0
3,999,3526,2021-06-24,3,100.0
4,999,13109,2021-05-15,3,100.0
5,999,14986,2021-06-26,3,100.0
6,999,7308,2021-04-25,3,100.0
7,999,5780,2021-04-21,3,100.0
8,999,6844,2021-07-03,3,100.0
9,999,830,2021-04-02,3,100.0


И последний аватар

In [104]:
documental_array = item_features.query('value == "документальное"')['id'].head(10).values

In [105]:
items[items['item_id'].isin(documental_array)]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre,country
31,10111,film,Андрей Тарковский. Кино как молитва,Andrey Tarkovsky. A Cinema Prayer,2019.0,документальное,"Россия, Италия, Швеция",NaN,12.0,NaN,Андрей А. Тарковский,NaN,Рассказ режиссера о самом себе. В основе фильм...,"Советский союз, тоскана, италия, портрет худож...",[документальное],"[россия, италия, швеция]"
50,3965,series,Уайат Сенак разрулит,Wyatt Cenac Problem Areas,2018.0,документальное,США,NaN,18.0,NaN,Уайат Сенак,Уайат Сенак,"Остроумное шоу о том, что нас окружает. В нем ...","Уайат, Сенак, разрулит, 2018, США",[документальное],[сша]
116,2753,film,"[4К] Уникальные скульптуры, искусство Асматов....","Unique sculptures, the Asmat art. West Papua",2020.0,документальное,Франция,NaN,12.0,NaN,Оливье Шиабоду,NaN,Поскольку племя асматов имеет воинское прошлое...,"2020, франция, 4к, уникальные, скульптуры, иск...",[документальное],[франция]
117,6731,film,[4К] Спокойные воды Бэ Д'юпи. Новая Каледония,Ubi bay,2020.0,документальное,Франция,NaN,12.0,NaN,Оливье Шиабоду,NaN,"Расположенный на севере острова Кутомо, залив ...",", 2020, франция, 4к, спокойные, воды, бэ, юпи,...",[документальное],[франция]
127,8832,film,Медитация орангутана,The orangutan's meditation,2020.0,документальное,Франция,NaN,12.0,NaN,Оливье Шиабоду,NaN,Орангутанг очень умён. Проживая большую часть ...,"2020, франция, медитация, орангутана",[документальное],[франция]
150,11015,film,Не укради. Возвращение святыни,NaN,2018.0,"исторические, русские, документальное",Россия,NaN,12.0,NaN,Николай Ахаян,NaN,"Документальный фильм о преступлении, произошед...","Не, укради, Возвращение, святыни, 2018, Россия...","[исторические, русские, документальное]",[россия]
178,345,film,Однажды... Тарантино,QT8: The First Eight,2019.0,"биография, документальное",США,NaN,18.0,NaN,Тара Вуд,"Зои Белл, Луис Блэк, Брюс Дерн, Роберт Форстер...",Откровенное путешествие в мир одного из самых ...,"кинобизнес, поп-культура, биография, восхожден...","[биография, документальное]",[сша]
183,15850,film,[4К] Вид сверху. ЮАР - На спине дракона,The view from above.South Africa - On the drag...,2018.0,документальное,Франция,NaN,12.0,NaN,Бернар Геррини,NaN,"Земля необъятных просторов и крайностей, заклю...","2018, франция, 4к, вид, сверху, юар, на, спине...",[документальное],[франция]
189,5210,film,Альдабра.Путешествие к таинственному острову,Aldabra: Once Upon an Island,2016.0,"семейное, приключения, документальное",Чехия,NaN,6.0,NaN,Стив Лихтаг,"Пирс Броснан, Ольдржих Кайзер",Присоединяйтесь к полной приключений 3D экспед...,"сейшельские острова, атолл альдабра, 2016, чех...","[семейное, приключения, документальное]",[чехия]
195,5091,film,[4К] Наветренные острова – жемчужина Полинезии,"Windwards Islands, Polynesian gem",2020.0,документальное,Франция,NaN,12.0,NaN,Оливье Шиабоду,NaN,Острова Общества и Наветренные острова являютс...,"2020, франция, 4к, наветренные, острова, жемчу...",[документальное],[франция]


In [106]:
inter_user_777 = pd.DataFrame({'user_id': [777]*10,
                               'item_id': documental_array,
                               'datetime': train['datetime'].sample(10).values,
                               'weight': [3]*10,
                               'watched_pct':[100.]*10}
                              )
inter_user_777

,user_id,item_id,datetime,weight,watched_pct
0,777,10111,2021-07-22,3,100.0
1,777,3965,2021-05-30,3,100.0
2,777,2753,2021-05-03,3,100.0
3,777,6731,2021-05-27,3,100.0
4,777,8832,2021-06-29,3,100.0
5,777,11015,2021-04-04,3,100.0
6,777,345,2021-06-13,3,100.0
7,777,15850,2021-04-17,3,100.0
8,777,5210,2021-07-08,3,100.0
9,777,5091,2021-06-07,3,100.0


Теперь объединим их с трейн выборкой и заново обучим модель, чтобы потом для них сделать предсказание

In [107]:
train = pd.concat((train, inter_user_666, inter_user_999, inter_user_777))
train = train.reset_index(drop=True)

Проверка

In [108]:
train

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,3,72.0
1,699317,1659,2021-05-29,3,100.0
2,656683,7107,2021-05-09,1,0.0
3,884009,693,2021-08-04,3,14.0
4,5324,8437,2021-04-18,3,92.0
...,...,...,...,...,...
922647,777,11015,2021-04-04,3,100.0
922648,777,345,2021-06-13,3,100.0
922649,777,15850,2021-04-17,3,100.0
922650,777,5210,2021-07-08,3,100.0


Заново собираем датасет и обучаем нашу модель, чтобы потом сделать предсказания

In [109]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", 'content_type', "country"],
)

In [110]:
test_users = np.array([666, 777, 999])

In [111]:
ALS.fit(dataset)
    
avatar_recos = ALS.recommend(
                      users=test_users,
                      dataset=dataset,
                      k=K_RECOS,
                      filter_viewed=True,
                      )

/usr/local/lib/python3.8/dist-packages/rectools/dataset/features.py:399: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")
/usr/local/lib/python3.8/dist-packages/rectools/models/implicit_als.py:269: UserWarning: GPU training requires number of factors to be a multiple of 32. Increasing factors from 292 to 320 (increasing latent factors from 96 to 124)
  warnings.warn(


 Посмотрим, что предсказала наша модель для наших аватаров

In [112]:
avatar_recos

,user_id,item_id,score,rank
0,666,9728,0.365489,1
1,666,13865,0.326971,2
2,666,10440,0.320992,3
3,666,15297,0.319803,4
4,666,4151,0.205994,5
5,666,3734,0.190816,6
6,666,6809,0.151366,7
7,666,142,0.132594,8
8,666,8636,0.129455,9
9,666,4740,0.101857,10


 Проверим, чтоже наша модель рекомендовала созданным аватарам

In [117]:
avatar_recos_user_666 = avatar_recos.loc[:9,'item_id'].values
avatar_recos_user_777 = avatar_recos.loc[10:19,'item_id'].values
avatar_recos_user_999 = avatar_recos.loc[20:29,'item_id'].values

In [118]:
items.query("item_id in @avatar_recos_user_999")

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre,country
544,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча...","[драмы, мелодрамы]",[россия]
1331,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе...","[триллеры, детективы]",[россия]
1335,7288,film,Шесть раз,Shesh Peamim,2012.0,"драмы, зарубежные",Израиль,NaN,18.0,NaN,Джонатан Гарфинкел,"Алон Лешем, Асаф Херц, Гита Амели, Джил Фишман...",«Шесть раз» – социальная драма израильского ре...,"Шесть, раз, 2012, Израиль, жизнь, тусовки, шко...","[драмы, зарубежные]",[израиль]
2749,6809,film,Дуров,NaN,2021.0,документальное,Россия,NaN,16.0,NaN,Родион Чепель,NaN,"Уникальная история о лидере нового поколения, ...","Компьютер, Монитор, Гений, Интервью, Предприни...",[документальное],[россия]
3301,8592,film,4 двойки,The Four Deuces,1975.0,"боевики, зарубежные, комедии",Израиль,NaN,12.0,NaN,NaN,"Адам Рорк, Бен Фроммер, Джанни Руссо, Джек Пэл...",США времён Великой депрессии. Вик Мороно – вла...,"4, двойки, 1975, Израиль, бандиты, гангстеры, ...","[боевики, зарубежные, комедии]",[израиль]
4405,13065,film,Повесть о любви и тьме,A Tale of Love and Darkness,2015.0,"драмы, зарубежные, биография",Израиль,NaN,16.0,NaN,Натали Портман,"Амир Теслер, Джилад Кахана, Йонатан Ширай, Мак...","История детства писателя Амоса Оза, выросшего ...","Повесть, о, любви, тьме, 2015, Израиль, жизнь,...","[драмы, зарубежные, биография]",[израиль]
4495,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер...","[боевики, триллеры]","[великобритания, сша]"
4604,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя...","[драмы, военные, приключения]",[россия]
4740,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо...",[комедии],[россия]
8521,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения",[комедии],[россия]


Как видно, модель рекомендуют полную чушь, точнее популярные айтемы для всех пользователей =) надо пересматривать модель и подход, или скорее всего создавать больше признаков для юзеров и айтемов, чтобы рекомендации стали более персонализированные

# Четвертая часть задания

- Придумать как можно обработать рекомендации для холодных пользователей. **(3 балла)**

Для холодных юзеров будем идти все тем же простым путем, и рекомендовать им список популярных айтемов

Функция для формирования списка айтемов за последние 14 дней

In [115]:
def popoular_number_of_items_days(
    df: pd.DataFrame, k: int = 10, days: int = 14, all_time: bool = False
) -> np.array:
    """
    Return a np.array of top@k most popular items for last N days
    """
    if all_time is True:
        recommendations = df.loc[:, "item_id"].value_counts().head(k).index.values
    else:
        min_date = df["datetime"].max().normalize() - pd.DateOffset(days)
        recommendations = (
            df.loc[df["datetime"] > min_date, "item_id"]
            .value_counts()
            .head(k)
            .index.values
        )
    return list(recommendations)

In [116]:
list_pop_items_14d = popoular_number_of_items_days(interactions, days=14)
list_pop_items_14d

[9728, 15297, 10440, 13865, 12192, 341, 7793, 3734, 4151, 14488]